## Deformable Registration

Processes already linearly registered data by applying deformable registration to it.
The fixed modality is SPECT and CT/PET get adjusted.

In [1]:
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt
from PIL import Image
from scipy import signal
import cv2

import os
import sys
import random

In [2]:
path = '/home/peter/Documents/dose_estimator-git/data/data_filtered/'
path_out = '/home/peter/Documents/dose_estimator-git/data/data_def_reg/'

In [3]:
# go through file system
pet = []
ct = []
dose= []

blacklist = ['06z1', '07z2', '12z4', '20z1', '23z1', '27z1', '62z1', '62z2']

for r, d, f in os.walk(path):
    for file in f:
        if '.nii.gz' in file and file[:4] not in blacklist:
            if 'pet' in file:
                pet.append(os.path.join(r, file))
            elif 'ct' in file:
                ct.append(os.path.join(r, file))
            else:
                dose.append(os.path.join(r, file))
print(dose[0])

/home/peter/Documents/dose_estimator-git/data/data_filtered/24z2.nii.gz


In [4]:
print(len(dose))

37


In [5]:
elastixImageFilter = sitk.ElastixImageFilter()

In [6]:
allx = len(dose)

for i, x in enumerate(dose):
    idx = x.split("/")[-1].split(".")[0]
    print(f"{i+1}/{allx}")
    fixed_img = sitk.ReadImage(os.path.join(path + f"{idx}.nii.gz"))
    elastixImageFilter.SetFixedImage(fixed_img)
    
    elastixImageFilter.SetMovingImage(sitk.ReadImage(os.path.join(path + f"{idx}_ct.nii.gz")))
    parameterMapVector = sitk.VectorOfParameterMap()
    parameterMapVector.append(sitk.GetDefaultParameterMap("affine"))
    parameterMapVector.append(sitk.GetDefaultParameterMap("bspline"))
    elastixImageFilter.SetParameterMap(parameterMapVector)
    elastixImageFilter.Execute()
    sitk.WriteImage(elastixImageFilter.GetResultImage(), os.path.join(path_out+f"{idx}_ct.nii.gz"), True)
    print(f"Processed {idx} CT image...")
    
    elastixImageFilter.SetMovingImage(sitk.ReadImage(os.path.join(path + f"{idx}_pet.nii.gz")))
    parameterMapVector = sitk.VectorOfParameterMap()
    parameterMapVector.append(sitk.GetDefaultParameterMap("affine"))
    parameterMapVector.append(sitk.GetDefaultParameterMap("bspline"))
    elastixImageFilter.SetParameterMap(parameterMapVector)
    elastixImageFilter.Execute()
    sitk.WriteImage(elastixImageFilter.GetResultImage(), os.path.join(path_out+f"{idx}_pet.nii.gz"), True)
    sitk.WriteImage(fixed_img, os.path.join(path_out+f"{idx}.nii.gz"), True)
    print(f"Processed {idx} PET image...")
    print("")

1/37
Processed 24z2 CT image...
Processed 24z2 PET image...

2/37
Processed 05z2 CT image...
Processed 05z2 PET image...

3/37
Processed 14z3 CT image...
Processed 14z3 PET image...

4/37
Processed 05z4 CT image...
Processed 05z4 PET image...

5/37
Processed 07z1 CT image...
Processed 07z1 PET image...

6/37
Processed 28z1 CT image...
Processed 28z1 PET image...

7/37
Processed 05z1 CT image...
Processed 05z1 PET image...

8/37
Processed 12z1 CT image...
Processed 12z1 PET image...

9/37
Processed 07z4 CT image...
Processed 07z4 PET image...

10/37
Processed 11z3 CT image...
Processed 11z3 PET image...

11/37
Processed 25z1 CT image...
Processed 25z1 PET image...

12/37
Processed 10z2 CT image...
Processed 10z2 PET image...

13/37
Processed 13z1 CT image...
Processed 13z1 PET image...

14/37
Processed 17z1 CT image...
Processed 17z1 PET image...

15/37
Processed 14z4 CT image...
Processed 14z4 PET image...

16/37
Processed 09z1 CT image...
Processed 09z1 PET image...

17/37
Processed 0

## Filter out completely black slices

In [9]:
def getDataset(train_path, test_path):
    path = train_path.rsplit("/",1)[0]+"/"
    print("TRAIN")
    ct_train, pet_train, dose_train = niftiToNumpy(train_path)
    np.save(os.path.join(path + 'ct_train.npy'), ct_train)
    np.save(os.path.join(path + 'pet_train.npy'), pet_train)
    np.save(os.path.join(path + 'dose_train.npy'), dose_train)
    del ct_train, pet_train, dose_train
    print("TEST")
    ct_test, pet_test, dose_test = niftiToNumpy(test_path)
    np.save(os.path.join(path + 'ct_test.npy'), ct_test)
    np.save(os.path.join(path + 'pet_test.npy'), pet_test)
    np.save(os.path.join(path + 'dose_test.npy'), dose_test)

In [12]:
def niftiToNumpy(path_to_file):
    text_file = open(path_to_file, "r", encoding='utf8')
    images = text_file.read().splitlines()
    
    path = path_to_file.rsplit("/",2)[0]+"/"
    
    ct = []
    pet = []
    dose = []
    
    allx = len(images)
    for i, idx in enumerate(images):
        print(f"Processing {i+1}/{allx}..")
        ct_img = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(path, f"{idx}_ct.nii.gz")))
        pet_img = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(path, f"{idx}_pet.nii.gz")))
        dose_img = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(path, f"{idx}.nii.gz")))
        
        if hasInformation(ct_img, pet_img, dose_img):
            ct.append(ct_img)
            pet.append(pet_img)
            dose.append(dose_img)
        
    ct = np.array(ct)
    pet = np.array(pet)
    dose = np.array(dose)
    return ct, pet, dose

In [4]:
def hasInformation(ct, pet, dose):
    ct_out = []
    pet_out = []
    dose_out = []
    for x in range(ct.shape[0]):
        if np.count_nonzero(ct[x]) == 0 or np.count_nonzero(pet[x]) == 0 or np.count_nonzero(dose[x]) == 0:
            ct_out.append(ct[x])
            pet_out.append(pet[x])
            dose_out.append(dose[x])
    ct_out = np.array(ct_out)
    pet_out = np.array(pet_out)
    dose_out = np.array(dose_out)
    return ct_out, pet_out, dose_out

In [13]:
train = '/home/peter/Documents/dose_estimator-git/data/data_filtered/numpy/train.txt'
test = '/home/peter/Documents/dose_estimator-git/data/data_filtered/numpy/test.txt'
getDataset(train, test)

TRAIN
Processing 1/32..
Processing 2/32..
Processing 3/32..
Processing 4/32..
Processing 5/32..
Processing 6/32..
Processing 7/32..
Processing 8/32..
Processing 9/32..
Processing 10/32..
Processing 11/32..
Processing 12/32..
Processing 13/32..
Processing 14/32..
Processing 15/32..
Processing 16/32..
Processing 17/32..
Processing 18/32..
Processing 19/32..
Processing 20/32..
Processing 21/32..
Processing 22/32..
Processing 23/32..
Processing 24/32..
Processing 25/32..
Processing 26/32..
Processing 27/32..
Processing 28/32..
Processing 29/32..
Processing 30/32..
Processing 31/32..
Processing 32/32..
TEST
Processing 1/5..
Processing 2/5..
Processing 3/5..
Processing 4/5..
Processing 5/5..
